In [ ]:
import pypsa
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from matplotlib.lines import Line2D
plt.style.use('bmh')
xr.set_options(display_style='html')
%matplotlib inline

In [ ]:
path = "../../../playgrounds/pr/"
clusters = 181
output = "../results/graphics/"

In [ ]:
regions = gpd.read_file(path + "pypsa-eur/resources_bk/regions_onshore_elec_s_181.geojson")

In [ ]:
n = pypsa.Network(path + "pypsa-eur/networks/elec.nc")

In [ ]:
len(n.buses)

In [ ]:
x = pypsa.Network(path + "pypsa-eur/networks/elec_s_181.nc")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), subplot_kw={"projection":ccrs.EqualEarth()})
x.plot(ax=ax, boundaries=[-9, 28, 72, 35])

handles = [
 Line2D([0], [0], color='rosybrown', lw=2),
 Line2D([0], [0], color='darkseagreen', lw=2),]
plt.legend(handles, ['HVAC', 'HVDC'], frameon=False, loc=[0.15,.85], fontsize=18)

plt.gca().outline_patch.set_visible(False)

plt.savefig(output + "network-s.pdf", bbox_inches='tight')

In [ ]:
len(n.buses.loc[n.buses.substation_lv==False])

In [ ]:
len(n.buses.loc[n.buses.substation_lv==True])

In [ ]:
len(n.lines)

In [ ]:
len(n.links)

In [ ]:
w = n.lines.v_nom.div(380)
c = n.lines.v_nom.map({220: 'teal', 300: 'orange', 380: 'firebrick'})

fig, ax = plt.subplots(subplot_kw={"projection":ccrs.EqualEarth()})
fig.set_size_inches(13,13)
n.plot(ax=ax, bus_sizes=0, line_widths=w, line_colors=c, link_colors='royalblue', link_widths=1, color_geomap=None)

handles = [Line2D([0], [0], color='teal', lw=2),
 Line2D([0], [0], color='orange', lw=2),
 Line2D([0], [0], color='firebrick', lw=2),
 Line2D([0], [0], color='royalblue', lw=2)]
plt.legend(handles, ['HVAC 220 kV', 'HVAC 300 kV', 'HVAC 380 kV', 'HVDC'], frameon=False, loc=[0.2,.85], fontsize=14)
plt.gca().outline_patch.set_visible(False)
plt.savefig(output + "network.pdf",bbox_inches='tight')

In [ ]:
regions = gpd.read_file(path + "pypsa-eur/resources/regions_onshore.geojson").append(
          gpd.read_file(path + "pypsa-eur/resources/regions_offshore.geojson"))
regions = regions.dissolve('name') 

In [ ]:
regions["Area"] = regions.to_crs(epsg=3035).area.div(1e6)

In [ ]:
onregions = gpd.read_file(path + "pypsa-eur/resources/regions_onshore.geojson").set_index('name')
onregions["Area"] = onregions.to_crs(epsg=3035).area.div(1e6)

In [ ]:
wind = pd.Series()
for profile in ['onwind', 'offwind-ac', 'offwind-dc']:
    ds = xr.open_dataset(path + f'pypsa-eur/resources/profile_{profile}.nc')
    wind = wind.append((ds.p_nom_max * ds.profile.sum('time')).to_pandas())
wind = wind.sum(level=0).reindex(regions.index, fill_value=0)
wind_per_skm = wind / regions.Area / 1e3 # GWh 

In [ ]:
proj = ccrs.EqualEarth()
regions = regions.to_crs(proj.proj4_init)
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection":proj})
regions.plot(ax=ax,column=wind_per_skm, cmap='Blues', linewidths=0,
           legend=True, legend_kwds={'label':r'Renewable Potential [GWh/a/km$^2$]', "shrink": 0.7})
ax.add_feature(cartopy.feature.COASTLINE.with_scale('50m'), linewidth=0.2, zorder=4) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')
plt.savefig(output + "wind-energy-density.pdf", bbox_inches='tight')

In [ ]:
ds = xr.open_dataset(path + 'pypsa-eur/resources/profile_solar.nc')
solar = (ds.p_nom_max * ds.profile.sum('time')).to_pandas()

solar = solar.sum(level=0).reindex(onregions.index, fill_value=0)
solar_per_skm = solar / onregions.Area / 1e3 # GWh

In [ ]:
proj = ccrs.EqualEarth()
onregions = onregions.to_crs(proj.proj4_init)
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection":proj})
onregions.plot(ax=ax,column=solar_per_skm, cmap='Reds', linewidths=0,
           legend=True, legend_kwds={'label':r'Renewable Potential [GWh/a/km$^2$]', "shrink": 0.7})
ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')
plt.savefig(output + "solar-energy-density.pdf", bbox_inches='tight')

In [ ]:
elec = pypsa.Network(path + "pypsa-eur/networks/elec.nc")

In [ ]:
wind = elec.generators_t.p_max_pu.filter(like='onwind').groupby(elec.generators.bus.map(elec.buses.country), axis=1).mean()["DE"]
solar = elec.generators_t.p_max_pu.filter(like='solar').groupby(elec.generators.bus.map(elec.buses.country), axis=1).mean()["DE"]

In [ ]:
sel = (wind.index >= "2013-02-01") & (wind.index < "2013-02-03") | \
    (wind.index >= "2013-05-01") & (wind.index < "2013-05-03") | \
    (wind.index >= "2013-08-01") & (wind.index < "2013-08-03") | \
    (wind.index >= "2013-11-01") & (wind.index < "2013-11-03")

In [ ]:
sel = (wind.index >= "2013-01-01") & (wind.index < "2014-01-01")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 3))

solar.where(sel).plot(ax=ax, c='#f9d002', alpha=0.5, linewidth=0.75, label="Germany Solar")
wind.where(sel).plot(ax=ax, c="#235ebc", label="Germany Wind")

plt.ylabel("Capacity Factor [-]")
plt.box(None)
plt.grid(False)
plt.legend(ncol=3, frameon=False, loc="upper center")
plt.ylim([0,1])
plt.xlabel(None)

plt.tight_layout()

plt.savefig(output + "temporal-fullyear.pdf", bbox_inches='tight')

In [ ]:
time = "03-2013"
i = elec.generators_t.p_max_pu.loc[time,"5709 onwind"]
c = elec.generators_t.p_max_pu.filter(like='onwind').groupby(elec.generators.bus.map(elec.buses.country), axis=1).mean().loc[time, "DE"]
a = elec.generators_t.p_max_pu.filter(like="onwind").mean(axis=1).loc[time]

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
plt.plot(i, label="Flensburg", linewidth=1.5, color="teal")
plt.plot(c, label="Germany",  linewidth=1.5, color="orange")
plt.plot(a, label="Europe",  linewidth=1.5, color="mediumpurple")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.xticks(rotation=30)
plt.ylabel("Capacity Factor [-]")
plt.box(None)
plt.grid(False)
plt.savefig(output + "ts-wind.pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
i.plot(ax=ax, label="Flensburg (Northern Germany)", linewidth=1.5, color="#235ebc")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.ylabel("Capacity Factor [-]")
plt.box(None)
plt.grid(False)
plt.xlabel(None)
plt.savefig(output + "ts-wind-reduced.pdf", bbox_inches='tight')

In [ ]:
#time = "01-03-2013":"05-03-2013"
start = "06-28-2013"
end = "07-07-2013"
i = elec.generators_t.p_max_pu.loc[start:end,"5709 solar"]
c = elec.generators_t.p_max_pu.filter(like='solar').groupby(elec.generators.bus.map(elec.buses.country), axis=1).mean().loc[start:end, "DE"]
a = elec.generators_t.p_max_pu.filter(like="solar").mean(axis=1).loc[start:end]

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
plt.plot(i, label="Flensburg", linewidth=1.5, color="teal")
plt.plot(c, label="Germany",  linewidth=1.5, color="orange")
plt.plot(a, label="Europe",  linewidth=1.5, color="mediumpurple")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.xticks(rotation=30)
plt.ylabel("Capacity Factor [-]")
plt.box(None)
plt.grid(False)
plt.savefig(output + "ts-solar.pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
i.plot(ax=ax, label="Flensburg (Northern Germany)", linewidth=1.5, color="#f9d002")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.ylabel("Capacity Factor [-]")
plt.box(None)
plt.grid(False)
plt.xlabel(None)
plt.savefig(output + "ts-solar-reduced.pdf", bbox_inches='tight')

In [ ]:
load = elec.loads_t.p_set.groupby(elec.loads.bus.map(elec.buses.country), axis=1).sum().div(1e3).loc["03-2013"]

In [ ]:
load.columns

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
plt.plot(load.ES, label="Spain", linewidth=1.5, color="teal")
plt.plot(load.DE, label="Germany",  linewidth=1.5, color="orange")
plt.plot(load.NO, label="Norway",  linewidth=1.5, color="mediumpurple")
#plt.plot(load.IE, label="Ireland",  linewidth=1.5, color="royalblue")
#plt.plot(load.IT, label="Estonia",  linewidth=1.5, color="darksalmon")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.xticks(rotation=30)
plt.ylabel("Electricity Load [GW]")
plt.box(None)
plt.grid(False)
plt.ylim([0,80])
plt.savefig(output + "ts-load.pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(5,2))
load.DE.plot(ax=ax,label="Germany",  linewidth=1.5, color="green")
plt.legend(ncol=3, frameon=False, loc=[0,1])
plt.ylabel("Electricity Load [GW]")
plt.box(None)
plt.grid(False)
plt.ylim([0,80])
plt.xlabel(None)
plt.savefig(output + "ts-load-reduced.pdf", bbox_inches='tight')

In [ ]:
smregions = gpd.read_file(path + "pypsa-eur/resources/regions_onshore.geojson").set_index('name')
elecs = pypsa.Network(path + "pypsa-eur/networks/elec_s.nc")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
smregions.plot(ax=ax, facecolor='steelblue', edgecolor='white')
plt.scatter(x=smregions.x, y=smregions.y, color='white', s=5)
plt.axis('off')
plt.ylim([52, 53])
plt.xlim([9,13])

In [ ]:
import atlite

In [ ]:
era5 = atlite.Cutout(path + "pypsa-eur/cutouts/europe-2013-era5.nc")

In [ ]:
sarah = atlite.Cutout(path + "pypsa-eur/cutouts/europe-2013-sarah.nc")

In [ ]:
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": ccrs.EqualEarth(), 'facecolor': "white"})

sarah.data.influx_direct.mean('time').plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='Oranges',
    linewidths=0,
    cbar_kwargs={'label':r'Mean Direct Solar Irradiance [W/m$^2$]', "shrink": 0.6}
)

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)

plt.savefig(output + "irradiation.png", dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": ccrs.EqualEarth(), 'facecolor': "white"})

era5.data.wnd100m.mean('time').plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='Blues',
    linewidths=0,
    cbar_kwargs={'label':r'Mean Wind Speeds [m/s]', "shrink": 0.7}
)


ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)

plt.savefig(output + "windspeeds.png", dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": ccrs.EqualEarth(), 'facecolor': "white"})

(era5.data.temperature.mean('time') - 273.15).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='Reds',
    linewidths=0,
    vmin=-5,
    vmax=30,
    cbar_kwargs={'label':r'Mean Temperatures [°C]', "shrink": 0.7}
)


ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)

plt.savefig(output + "temperatures.png", dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": ccrs.EqualEarth(), 'facecolor': "white"})

era5.data.runoff.sum('time').plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='Greens',
    linewidths=0,
    vmin=0,
    vmax=2,
    cbar_kwargs={'label':r'Total Runoff [?]', "shrink": 0.7}
)


ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
plt.gca().outline_patch.set_visible(False)

plt.savefig(output + "runoff.png", dpi=300, bbox_inches='tight')